**Q3. Given a bank customer, build a neural network-based classifier that can determine whether they will leave or not in the next 6 months. Dataset Description: The case study is from an open-source dataset from Kaggle. The dataset contains 10,000 sample points with 14 distinct features such as CustomerId, CreditScore, Geography, Gender, Age, Tenure, Balance, etc. Link to the Kaggle project: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling Perform following steps:**
1. Read the dataset.
2. Distinguish the feature and target set and divide the data set into training and test sets.
3. Normalize the train and test data.
4. Initialize and build the model. Identify the points of improvement and implement the same.
5. Print the accuracy score and confusion matrix (5 points).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Pre-processing

In [4]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
# convert categorical variables to numerical values
label_encoder = LabelEncoder()

# apply label encoding on 'Gender' and 'Geography' columns
df['Gender'] = label_encoder.fit_transform(df['Gender'])  # Female=0, Male=1
df['Geography'] = label_encoder.fit_transform(df['Geography'])  # France=0, Germany=1, Spain=2

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
# separate the features and the target variables
x = df.drop('Exited', axis = 1)
y = df['Exited']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [11]:
# normalize the dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
# we use only transform on x_test as it uses the same mean and SD learned from the training set

### Build the neural network

In [13]:
model = Sequential()
model.add(Dense(activation='relu', units=32, input_dim=x_train.shape[1]))  # input layer with more neurons
model.add(Dropout(0.5))  # dropout layer to prevent overfitting
model.add(Dense(activation='relu', units=16))  # second hidden layer
model.add(Dropout(0.5))
model.add(Dense(activation='sigmoid', units=1))  # output layer for binary classification

In [14]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train, batch_size=32, epochs=50)

Epoch 1/50
235/235 [==============================] - 1s 1ms/step - loss: 0.5637 - accuracy: 0.7436
Epoch 2/50
235/235 [==============================] - 0s 1ms/step - loss: 0.5026 - accuracy: 0.7944
Epoch 3/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4920 - accuracy: 0.7952
Epoch 4/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4711 - accuracy: 0.8007
Epoch 5/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4626 - accuracy: 0.8039
Epoch 6/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4622 - accuracy: 0.8059
Epoch 7/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4527 - accuracy: 0.8095
Epoch 8/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4514 - accuracy: 0.8107
Epoch 9/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4403 - accuracy: 0.8141
Epoch 10/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4399 - accuracy: 0.8123

In [16]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)  # convert the probabilistic values to True or False
y_pred

79/79 [==============================] - 0s 950us/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [17]:
# calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

# confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Print the results
print("Accuracy Score:", accuracy)
print("Confusion Matrix:\n", cm)

Accuracy Score: 0.8576
Confusion Matrix:
 [[1966   45]
 [ 311  178]]
